## Simple Reflection Architecture

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph, MessageGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os

from IPython.display import Image, display

import concurrent.futures

In [34]:
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o",
)

class PromptUpdate(BaseModel):
    """Review of the prompt"""
    updated_prompt: str = Field(description="Updated prompt based on the review")
    justification: str = Field(description="Justification for the changes made")

def self_reflection_graph(additional_info) -> MessageGraph:
    """
    Constructs a graph for self-reflection and improvement of prompts.
    """

    system_message = SystemMessage(content=f"""You are an experienced: AI prompt engineer. Your core principles are:
- Always write clear and concise prompts.
- Always write contextually relevant prompts.
- Always write task aligned prompts.
- Always write example demonstrations in prompts.
- Always format and structure prompts to be easily understood by the model.
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")

    def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptUpdate)
        prompt_text = """Your task is to improve the prompt in light of your core principles.
If the user provides critique and recommendations for the prompt, respond with a revised version of your previous attempts.
You must provide justification, in less than 100 words, for any changes you make to the prompt.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.

Your update process should be as follows:
1. Read the prompt carefully asn an expert AI engineer.
2. Think carefully about how you can implement the user's feedback.
3. Implement the changes to the prompt.
4. Ensure that your changes adhere to the strict guidelines.
5. Provide justification for the changes you made.
6. Ensure that your justification is less than 50 words.
7. Submit your updated prompt and justification.

Return the updated prompt along with your review in JSON format below. You must not output your review.

{{
"updated_prompt": "updated prompt based on the review",
"justification": "Justification for the changes made"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly.
"""
        prompt = HumanMessagePromptTemplate(
            prompt = PromptTemplate(
                system_message=system_message,
                template=prompt_text,
                input_variables=["additional_info"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
            )
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                prompt,
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
        for _ in range(3):
            try:
                generate = prompt | llm | pydantic_parser
                res = generate.invoke({"messages": state, "additional_info": additional_info})
                updated_prompt_message = AIMessage(content=res.updated_prompt)
                justification_message = AIMessage(content=res.justification)
                new_state = state + [updated_prompt_message, justification_message]
                print(f"New state: {new_state}")
                return new_state
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")
        
    def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
        reflection_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    f"""{system_message.content} 
Your task is to think outside the box to provide creative feedback with recommendations on how to improve the user's prompt in light of your core principles.
Your feedback must be less than 100 words so think carefully about the most critical aspects of the prompt that need improvement.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow when providing feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert AI engineer. 
2. Identify the most critical aspects of the prompt that need improvement. 
3. Think outside the box to provide creative feedback with recommendations on how to improve the prompt in light of your core principles.
4. Ensure that your feedback is less than 100 words.
5. Ensure that your feedback follows the strict guidelines provided above.
6. Submit your feedback.
""",
                ),
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
        reflect = reflection_prompt | llm
        res = reflect.invoke({"messages": messages})
        # We treat the output of this as human feedback for the generator
        return HumanMessage(content=res.content)

    builder = MessageGraph()
    builder.add_node("generate", generation_node)
    builder.add_node("reflect", reflection_node)
    builder.set_entry_point("generate")

    def should_continue(state: List[BaseMessage]):
        if len(state)/2 > 10:
            return END
        return "reflect"

    builder.add_conditional_edges("generate", should_continue)
    builder.add_edge("reflect", "generate")
    graph = builder.compile()

    return graph

def update_prompt(prompt: str, additional_info: str) -> str:
    """
    Uses self_reflection_graph to iteratively act on feedback and update prompt
    """
    graph = self_reflection_graph(additional_info)
    inputs = [HumanMessage(content=prompt)]
    for event in graph.stream(
        inputs,
        {"recursion_limit": 100}
    ):
        print(event)
        print("----")
    return event

In [35]:
base_prompt = "{content}. Please output your answer at the end as ##<your answer (arabic numerals)>."
additional_info = "- Solve the math word problem.\n- Output the answer at the end as ##<your answer (arabic numerals)> with no spaces or units."

prompt = update_prompt(base_prompt, additional_info)

New state: [HumanMessage(content='{content}. Please output your answer at the end as ##<your answer (arabic numerals)>.', id='17c63285-0aed-4b52-9a4a-3eed63918ddb'), AIMessage(content='{content}. Please solve the math word problem and output your answer at the end as ##<your answer (arabic numerals)>.'), AIMessage(content="Added 'solve the math word problem' to align with the task's requirement and ensure clarity.")]
{'generate': [HumanMessage(content='{content}. Please output your answer at the end as ##<your answer (arabic numerals)>.', id='17c63285-0aed-4b52-9a4a-3eed63918ddb'), AIMessage(content='{content}. Please solve the math word problem and output your answer at the end as ##<your answer (arabic numerals)>.', id='620d5fa4-575f-4619-bdf7-f5d8ff1896b5'), AIMessage(content="Added 'solve the math word problem' to align with the task's requirement and ensure clarity.", id='81644b1b-dbab-441e-8856-fe19c5d6589a')]}
----
{'reflect': HumanMessage(content='The prompt should explicitly i

In [36]:
prompt["generate"][-2].content

'Please solve the math word problem in {content} and output your answer at the end as ##<your answer (arabic numerals)>.'